In [1]:
import search_model
import surrogate_model

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import argparse

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
args = argparse.Namespace(
    target = ['strength','cement'],
    data_path='./data/concrete_processed.csv',
    model='catboost',
    prj_id = 41,
    seed=41
)
surrogate_model.main(args)

2025-02-01 16:57:54,676 - INFO - X_train.shape: (740, 7)
2025-02-01 16:57:54,677 - INFO - X_test.shape: (186, 7)
2025-02-01 16:57:54,677 - INFO - y_train.shape: (740, 2)
2025-02-01 16:57:54,678 - INFO - y_test.shape: (186, 2)
[I 2025-02-01 16:57:54,678] A new study created in memory with name: no-name-1899528b-3907-475c-8648-da67179e2fd9
[I 2025-02-01 16:57:54,921] Trial 0 finished with value: 0.017495474216852217 and parameters: {'objective': 'MultiRMSE', 'colsample_bylevel': 0.04026047729395524, 'depth': 8}. Best is trial 0 with value: 0.017495474216852217.
[I 2025-02-01 16:57:55,104] Trial 1 finished with value: 0.015525342800904373 and parameters: {'objective': 'MultiRMSE', 'colsample_bylevel': 0.053962407472888975, 'depth': 2}. Best is trial 1 with value: 0.015525342800904373.
[I 2025-02-01 16:57:55,322] Trial 2 finished with value: 0.008603321628052212 and parameters: {'objective': 'MultiRMSE', 'colsample_bylevel': 0.0855369378962201, 'depth': 12}. Best is trial 2 with value: 0.0

FrozenTrial(number=94, state=TrialState.COMPLETE, values=[0.007509979129136569], datetime_start=datetime.datetime(2025, 2, 1, 16, 58, 14, 920031), datetime_complete=datetime.datetime(2025, 2, 1, 16, 58, 15, 162494), params={'objective': 'MultiRMSE', 'colsample_bylevel': 0.09786028946137411, 'depth': 8}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'objective': CategoricalDistribution(choices=('MultiRMSE',)), 'colsample_bylevel': FloatDistribution(high=0.1, log=False, low=0.01, step=None), 'depth': IntDistribution(high=12, log=False, low=1, step=1)}, trial_id=94, value=None)
{'objective': 'MultiRMSE', 'colsample_bylevel': 0.09786028946137411, 'depth': 8}
0.007509979129136569
0:	learn: 0.3178507	total: 271us	remaining: 271ms
1:	learn: 0.3178507	total: 387us	remaining: 193ms
2:	learn: 0.3166098	total: 637us	remaining: 212ms
3:	learn: 0.3151360	total: 860us	remaining: 214ms
4:	learn: 0.3133751	total: 1.12ms	remaining: 223ms
5:	learn: 0.3133751	total: 1.24ms	remain

In [5]:
import os

In [6]:
os.listdir('./prj/41/surrogate_model')

['model.txt', 'eval.csv', 'surrogate_model.csv']